<a href="https://colab.research.google.com/github/esleylab/machinelearning/blob/main/learningtransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

import tensorflow_datasets as tfds

# Baixando o dataset Cats vs Dogs
dataset_name = "cats_vs_dogs"
data, info = tfds.load(dataset_name, as_supervised=True, with_info=True)

# Dividindo em treino e teste
train_data = data['train'].take(int(info.splits['train'].num_examples * 0.8)) # Seleciona 80% dos dados de treino
test_data = data['train'].skip(int(info.splits['train'].num_examples * 0.2)) # Skip 80% para selecionar os 20% restantes para teste
processed = False

# Informações do dataset
#print(info)


In [ ]:
import tensorflow as tf

# Função de preprocessamento
def preprocess(image, label):
    image = tf.image.resize(image, (128, 128))  # Redimensiona para diminuir processamento
    image = image / 255.0  # Normaliza para o intervalo [0, 1]
    return image, label

# Aplicar preprocessamento e criar batches
if not processed:
  batch_size = 32
  train_data = train_data.map(preprocess).batch(batch_size).shuffle(1000)
  test_data = test_data.map(preprocess).batch(batch_size)
  processed = True #Controle para não processar 2x

# Visualizando um lote de imagens
import matplotlib.pyplot as plt

for images, labels in train_data.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title("Gato" if labels[i] == 0 else "Cachorro")
        plt.axis("off")
    plt.show()